In [84]:
import os 
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import xarray as xr
import seaborn as sns
from pathlib import Path
import datetime
import plotly.express as px
import dataframe_image as dfi
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio
from matplotlib.ticker import MaxNLocator

# REMI Datei

In [168]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df_remit = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_remit.head(3)

##### Erstellung der Standertwerte

In [ ]:
energy_null = df_remit.isnull().sum()

df_energy_null = pd.DataFrame(energy_null).reset_index()
df_energy_null.columns = ['Feature', 'Null Count'] 



display(df_energy_null)
# df_energy_null = pd.DataFrame(df_energy_null).reset_index()
# df_energy_null.columns = ['Feature', 'Null Count'] 



# display(df_energy_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
      
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

      
        png_filename = f"{df_name}.png"

     
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        
        png_full_path = f"{path}/{png_filename}"

       
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')


export_styled_dataframes(dataframes, path=r'...')



## Energy Data

In [88]:
energy_20200920_20231027  = pd.read_csv(r"data\Energy_Data_20200920_20231027.csv")
energy_20200920_20240118  = pd.read_csv(r"data\Energy_Data_20200920_20240118.csv")
energy_20240119_20240519  = pd.read_csv(r"data\Energy_Data_20240119_20240519.csv")

In [89]:
energy = pd.concat([energy_20200920_20240118, energy_20240119_20240519], ignore_index=True)

In [90]:
energy_h3 = energy.head(3).round(2)

##### shape

In [ ]:
energy.shape

##### Duplikate

In [ ]:
dupes = energy[energy.duplicated()]

print("Duplikate:")
print(dupes.shape)

##### Erstellung der Standertwerte


In [ ]:
energy_null = energy.isnull().sum()

df_energy_null = pd.DataFrame(energy_null).reset_index()
df_energy_null.columns = ['Feature', 'Null Count'] 



display(df_energy_null)


def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
      
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

      
        png_filename = f"{df_name}.png"

     
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        
        png_full_path = f"{path}/{png_filename}"

       
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = energy.min()
max_values = energy.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)

merged_df_standard = pd.merge(df_energy_null, min_max_df, on='Feature', how='inner')

merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'dtm']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(['dtm'])
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)





dataframes = {
    'df_energy_data_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


### Data Analysis

In [ ]:
num_energy_cols = energy.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_energy_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.histplot(data=num_energy_cols, x=num_energy_cols.columns[i])
    plt.title(num_energy_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
num_energy_cols = energy.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_energy_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.boxplot(data=num_energy_cols, x=num_energy_cols.columns[i])
    plt.title(num_energy_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
corr = energy.select_dtypes(include="number").corr()
sns.heatmap(corr, annot=True)

## Weather data

### DWD hornsea 1 

In [98]:
def load_hornsea(dataset: xr.Dataset) -> pd.DataFrame:
    df = (
        dataset
        .mean(dim=["latitude", "longitude"])
        .to_dataframe()
        .reset_index()
    ).rename(columns={"ref_datetime": "reference_time", "valid_datetime": "valid_time"})
    df = (
        df.assign(
            reference_time=df["reference_time"].dt.tz_localize("UTC"),
            hours_after=df["valid_time"],
            valid_time=(df["reference_time"] + pd.to_timedelta(df["valid_time"], unit="hours")).dt.tz_localize("UTC")
        )
    )
    return df


In [99]:
dwd_hornsea_1 = load_hornsea(xr.load_dataset(r"data\dwd_icon_eu_hornsea_1_20200920_20231027.nc", engine="h5netcdf"))
dwd_hornsea_2 = load_hornsea(xr.load_dataset(r"data\dwd_icon_eu_hornsea_1_20231027_20240108.nc", engine="h5netcdf"))
dwd_hornsea_3 = load_hornsea(xr.load_dataset(r"data\dwd_icon_eu_hornsea_1_20240108_20240129.nc", engine="h5netcdf"))
dwd_hornsea_4 = load_hornsea(xr.load_dataset(r"data\dwd_icon_eu_hornsea_1_20240129_20240519.nc", engine="h5netcdf"))

In [100]:
df_hornsea_dwd = (pd.concat([dwd_hornsea_1, dwd_hornsea_2, dwd_hornsea_3, dwd_hornsea_4])
              .sort_values(["reference_time", "valid_time"])
              .reset_index(drop=True))

In [101]:
df_hornsea_dwd_h3 = df_hornsea_dwd.head(3).round(2)

In [ ]:
df_hornsea_dwd.describe().T

##### shape

In [ ]:
df_hornsea_dwd.shape

##### Duplikate

In [ ]:
dupes = df_hornsea_dwd[df_hornsea_dwd.duplicated()]

print("Duplikate:")
print(dupes.shape)

In [ ]:
df_hornsea_dwd[df_hornsea_dwd['RelativeHumidity']>100]

##### Erstellung der Standertwerte

In [ ]:
df_hornsea_dwd_null = df_hornsea_dwd.isnull().sum()
df_hornsea_dwd_null

df_hornsea_dwd_null = pd.DataFrame(df_hornsea_dwd_null).reset_index()
df_hornsea_dwd_null.columns = ['Feature', 'Null Count'] 



display(df_hornsea_dwd_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
      
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

      
        png_filename = f"{df_name}.png"

     
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        
        png_full_path = f"{path}/{png_filename}"

       
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_hornsea_dwd.min()
max_values = df_hornsea_dwd.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)

merged_df_standard = pd.merge(df_hornsea_dwd_null, min_max_df, on='Feature', how='inner')

merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'reference_time','valid_time']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)



dataframes = {
    'df_dwd_hornsea_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


### Data Analysis

In [ ]:
num_dwd_hornsea_cols = df_hornsea_dwd.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_dwd_hornsea_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.histplot(data=num_dwd_hornsea_cols, x=num_dwd_hornsea_cols.columns[i])
    plt.title(num_dwd_hornsea_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
num_dwd_hornsea_cols = df_hornsea_dwd.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_dwd_hornsea_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.boxplot(data=num_dwd_hornsea_cols, x=num_dwd_hornsea_cols.columns[i])
    plt.title(num_dwd_hornsea_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
corr = df_hornsea_dwd.select_dtypes(include="number").corr()
sns.heatmap(corr, annot=True)

## pes10 data

In [110]:

def pre_dwd(df):
    # Konvertiert ein xarray Dataset in einen DataFrame und setzt den Index zurück
    df = df.to_dataframe().reset_index()
    # Lokalisiert 'ref_datetime' auf UTC
    df.ref_datetime = df.ref_datetime.dt.tz_localize("UTC")
    # Berechnet 'valid_datetime' basierend auf 'ref_datetime' und einer Zeitspanne in Stunden
    df.valid_datetime = df.ref_datetime + df.valid_datetime * pd.Timedelta(1, "h")
    return df


In [ ]:
df_dwd_pes10_1 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc")).rename(
        # {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_pes10_2 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_pes10_20231027_20240108.nc")).rename(
         {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_pes10_3 = pre_dwd(
    (dataset := xr.load_dataset("data/ncep_gfs_pes10_20240108_20240129.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_pes10_4 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_pes10_20240129_20240519.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)

df_dwd_pes10 = (pd.concat([df_dwd_pes10_1, df_dwd_pes10_2, df_dwd_pes10_3, df_dwd_pes10_4])
              .sort_values(["ref_datetime", "valid_datetime"])
              .reset_index(drop=True))
df_dwd_pes10.head(2)


In [ ]:
df_dwd_pes10.head(50)

In [113]:
df_dwd_pes10_h3 = df_dwd_pes10.head(3).round(2)

##### shape

In [ ]:
df_dwd_pes10.shape

##### Duplikate

In [ ]:
dupes = df_dwd_pes10[df_dwd_pes10.duplicated()]

print("Duplikate:")
print(dupes.shape)

##### Erstellung der Standertwerte

In [ ]:
df_dwd_pes10_null = df_dwd_pes10.isnull().sum()
df_dwd_pes10_null
# In ein DataFrame umwandeln
df_dwd_pes10_null = pd.DataFrame(df_dwd_pes10_null).reset_index()
df_dwd_pes10_null.columns = ['Feature', 'Null Count']  # Spalten umbenennen


# Ausgabe des neuen DataFrames
display(df_dwd_pes10_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        # Überprüfen, ob df ein DataFrame ist
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

        # Dateiname für den DataFrame
        png_filename = f"{df_name}.png"

        # Styling-Optionen definieren
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        # Vertikale und horizontale Linien für jede Zelle hinzufügen
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        # Dateipfad und -namen kombinieren
        png_full_path = f"{path}/{png_filename}"

        # DataFrame als Bild speichern mit dfi
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_dwd_pes10.min()
max_values = df_dwd_pes10.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)
# df_dwd_pes10_null.merge(min_max_df,on = 'Feature', how='inner')
merged_df_standard = pd.merge(df_dwd_pes10_null, min_max_df, on='Feature', how='inner')

merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'ref_datetime','valid_datetime']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)

# Korrigierter DataFrame-Namen
dataframes = {
    'df_dwd_pes10_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'C:\Users\Michael Jäckle\Desktop\00_Inbox\Now\00_Bilder_Doku')
display(merged_df_standard)


## demand

In [117]:
def pre_csv(df):
    # Wandelt die Spalte 'dtm' in datetime-Objekte um
    df.dtm = pd.to_datetime(df.dtm)
    return df

def pre_dwd(df):
    # Konvertiert ein xarray Dataset in einen DataFrame und setzt den Index zurück
    df = df.to_dataframe().reset_index()
    # Lokalisiert 'ref_datetime' auf UTC
    df.ref_datetime = df.ref_datetime.dt.tz_localize("UTC")
    # Berechnet 'valid_datetime' basierend auf 'ref_datetime' und einer Zeitspanne in Stunden
    df.valid_datetime = df.ref_datetime + df.valid_datetime * pd.Timedelta(1, "h")
    return df

def pre_ncep(df):
    return df


In [118]:
df_dwd_demand_1 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_demand_20200920_20231027.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_demand_2 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_demand_20231027_20240108.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_demand_3 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_demand_20240108_20240129.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)
df_dwd_demand_4 = pre_dwd(
    (dataset := xr.load_dataset("data/dwd_icon_eu_demand_20240129_20240519.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset}
    )
)

df_dwd_demand = (pd.concat([df_dwd_demand_1, df_dwd_demand_2, df_dwd_demand_3, df_dwd_demand_4])
              .sort_values(["ref_datetime", "valid_datetime"])
              .reset_index(drop=True))


In [ ]:
df_dwd_demand

In [ ]:
df_dwd_demand_2.head()

In [121]:
df_dwd_demand_h3 = df_dwd_demand.head(3).round(2)

##### shape

In [ ]:
df_dwd_demand.shape

##### Duplikate

In [ ]:
dupes = df_dwd_demand[df_dwd_demand.duplicated()]

print("Duplikate:")
print(dupes.shape)

In [ ]:
df_dwd_demand.isnull().sum()

##### Erstellung der Standertwerte

In [ ]:
df_dwd_demand_null = df_dwd_demand.isnull().sum()
df_dwd_demand_null
# In ein DataFrame umwandeln
df_dwd_demand_null = pd.DataFrame(df_dwd_demand_null).reset_index()
df_dwd_demand_null.columns = ['Feature', 'Null Count']  # Spalten umbenennen


# Ausgabe des neuen DataFrames
display(df_dwd_demand_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        # Überprüfen, ob df ein DataFrame ist
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

        # Dateiname für den DataFrame
        png_filename = f"{df_name}.png"

        # Styling-Optionen definieren
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        # Vertikale und horizontale Linien für jede Zelle hinzufügen
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        # Dateipfad und -namen kombinieren
        png_full_path = f"{path}/{png_filename}"

        # DataFrame als Bild speichern mit dfi
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_dwd_demand.min()
max_values = df_dwd_demand.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)
# df_dwd_pes10_null.merge(min_max_df,on = 'Feature', how='inner')
merged_df_standard = pd.merge(df_dwd_demand_null, min_max_df, on='Feature', how='inner')
merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'ref_datetime','valid_datetime']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)

# Korrigierter DataFrame-Namen
dataframes = {
    'df_dwd_demand_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


In [ ]:
100/ 3441186*76615

### NCEP Hornsea

In [127]:
ncep_hornsea_1 = load_hornsea(xr.load_dataset(r"data\ncep_gfs_hornsea_1_20200920_20231027.nc", engine="h5netcdf"))
ncep_hornsea_2 = load_hornsea(xr.load_dataset(r"data\ncep_gfs_hornsea_1_20231027_20240108.nc", engine="h5netcdf"))
ncep_hornsea_3 = load_hornsea(xr.load_dataset(r"data\ncep_gfs_hornsea_1_20240108_20240129.nc", engine="h5netcdf"))
ncep_hornsea_4 = load_hornsea(xr.load_dataset(r"data\ncep_gfs_hornsea_1_20240129_20240519.nc", engine="h5netcdf"))

In [128]:
df_hornsea_ncep = (pd.concat([ncep_hornsea_1, ncep_hornsea_2, ncep_hornsea_3, ncep_hornsea_4])
              .sort_values(["reference_time", "valid_time"])
              .reset_index(drop=True))

In [129]:
df_hornsea_ncep_h3 = df_hornsea_ncep.head(3).round(2)

In [ ]:
df_hornsea_ncep.describe().T

### Data Analysis

In [ ]:
num_hornsea_ncep_cols = df_hornsea_ncep.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_hornsea_ncep_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.histplot(data=num_hornsea_ncep_cols, x=num_hornsea_ncep_cols.columns[i])
    plt.title(num_hornsea_ncep_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
num_hornsea_ncep_cols = df_hornsea_ncep.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_hornsea_ncep_cols.columns)):
    plt.subplot(3, 3, i+1)
    sns.boxplot(data=num_hornsea_ncep_cols, x=num_hornsea_ncep_cols.columns[i])
    plt.title(num_hornsea_ncep_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [133]:
df_ncep_hornsea_h3 = df_hornsea_ncep.head(3).round(2)

##### shape

In [ ]:
df_hornsea_ncep.shape

##### Duplikate

In [ ]:
dupes = df_hornsea_ncep[df_hornsea_ncep.duplicated()]

print("Duplikate:")
print(dupes.shape)

##### Erstellung der Standertwerte

In [ ]:
df_hornsea_ncep_null = df_hornsea_ncep.isnull().sum()
df_hornsea_ncep_null
# In ein DataFrame umwandeln
df_hornsea_ncep_null = pd.DataFrame(df_hornsea_ncep_null).reset_index()
df_hornsea_ncep_null.columns = ['Feature', 'Null Count']  # Spalten umbenennen


# Ausgabe des neuen DataFrames
display(df_hornsea_ncep_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        # Überprüfen, ob df ein DataFrame ist
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

        # Dateiname für den DataFrame
        png_filename = f"{df_name}.png"

        # Styling-Optionen definieren
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        # Vertikale und horizontale Linien für jede Zelle hinzufügen
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        # Dateipfad und -namen kombinieren
        png_full_path = f"{path}/{png_filename}"

        # DataFrame als Bild speichern mit dfi
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_hornsea_ncep.min()
max_values = df_hornsea_ncep.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)
# df_dwd_pes10_null.merge(min_max_df,on = 'Feature', how='inner')
merged_df_standard = pd.merge(df_hornsea_ncep_null, min_max_df, on='Feature', how='inner')

merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'reference_time','valid_time']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)
# Korrigierter DataFrame-Namen
dataframes = {
    'df_ncep_hornsea_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


In [ ]:
corr = df_hornsea_ncep.select_dtypes(include="number").corr()
sns.heatmap(corr, annot=True)

##  ncep_gfs_demand

In [138]:
def pre_ncep(df):
    return df

In [139]:
df_ncep_demand_1 = pre_ncep(xr.open_dataset("data/ncep_gfs_demand_20200920_20231027.nc"))
df_ncep_demand_1 = df_ncep_demand_1.to_dataframe().reset_index()
df_ncep_demand_2 = pre_ncep(xr.open_dataset("data/ncep_gfs_demand_20231027_20240108.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_demand_2 = df_ncep_demand_2.to_dataframe().reset_index()
df_ncep_demand_3 = pre_ncep(xr.open_dataset("data/ncep_gfs_demand_20240108_20240129.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_demand_3 = df_ncep_demand_3.to_dataframe().reset_index()
df_ncep_demand_4 = pre_ncep(xr.open_dataset("data/dwd_icon_eu_demand_20240129_20240519.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_demand_4 = df_ncep_demand_4.to_dataframe().reset_index()

In [140]:
df_ncep_demand = (pd.concat([df_ncep_demand_1, df_ncep_demand_2, df_ncep_demand_3, df_ncep_demand_4])
              .sort_values(["ref_datetime", "valid_datetime"])
              .reset_index(drop=True))

In [ ]:
df_ncep_demand_h3 = df_ncep_demand.head(3).round(2)
df_ncep_demand_h3

##### shape

In [ ]:
df_ncep_demand.shape

#### Duplikate

In [ ]:
dupes = df_ncep_demand[df_ncep_demand.duplicated()]

print("Duplikate:")
print(dupes.shape)

##### Erstellung der Standertwerte

In [ ]:
df_ncep_demand_null = df_ncep_demand.isnull().sum()
df_ncep_demand_null
# In ein DataFrame umwandeln
df_ncep_demand_null = pd.DataFrame(df_ncep_demand_null).reset_index()
df_ncep_demand_null.columns = ['Feature', 'Null Count']  # Spalten umbenennen


# Ausgabe des neuen DataFrames
display(df_ncep_demand_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        # Überprüfen, ob df ein DataFrame ist
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

        # Dateiname für den DataFrame
        png_filename = f"{df_name}.png"

        # Styling-Optionen definieren
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        # Vertikale und horizontale Linien für jede Zelle hinzufügen
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        # Dateipfad und -namen kombinieren
        png_full_path = f"{path}/{png_filename}"

        # DataFrame als Bild speichern mit dfi
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_ncep_demand.min()
max_values = df_ncep_demand.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)
# df_dwd_pes10_null.merge(min_max_df,on = 'Feature', how='inner')
merged_df_standard = pd.merge(df_ncep_demand_null, min_max_df, on='Feature', how='inner')


merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'ref_datetime','valid_datetime']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)


# Korrigierter DataFrame-Namen
dataframes = {
    'df_ncep_demand_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


## df_ncep_pes10

In [145]:
def pre_ncep(df):
    return df

In [146]:
df_ncep_pes10_1 = pre_ncep(xr.open_dataset("data/ncep_gfs_pes10_20200920_20231027.nc"))
df_ncep_pes10_1 = df_ncep_pes10_1.to_dataframe().reset_index()
df_ncep_pes10_2 = pre_ncep(xr.open_dataset("data/ncep_gfs_pes10_20231027_20240108.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_pes10_2 = df_ncep_pes10_2.to_dataframe().reset_index()
df_ncep_pes10_3 = pre_ncep(xr.open_dataset("data/ncep_gfs_pes10_20240108_20240129.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_pes10_3 = df_ncep_pes10_3.to_dataframe().reset_index()
df_ncep_pes10_4 = pre_ncep(xr.open_dataset("data/ncep_gfs_pes10_20240129_20240519.nc")).rename(
        {old: new for old, new in {'reference_time': 'ref_datetime', 'valid_time': 'valid_datetime'}.items() if old in dataset})
df_ncep_pes10_4 = df_ncep_pes10_4.to_dataframe().reset_index()

In [147]:
df_ncep_pes10 = (pd.concat([df_ncep_pes10_1, df_ncep_pes10_2, df_ncep_pes10_3, df_ncep_pes10_4])
              .sort_values(["ref_datetime", "valid_datetime"])
              .reset_index(drop=True))

In [ ]:
df_ncep_pes10_h3 = df_ncep_pes10.head(3).round(2)
df_ncep_pes10.head(50)

##### shape

In [ ]:
df_ncep_pes10.shape

##### Duplikate

In [ ]:
dupes = df_ncep_pes10[df_ncep_pes10.duplicated()]

print("Duplikate:")
print(dupes.shape)

##### Erstellung der Standertwerte

In [ ]:
df_ncep_pes10_null = df_ncep_pes10.isnull().sum()
df_ncep_pes10_null
# In ein DataFrame umwandeln
df_ncep_pes10_null = pd.DataFrame(df_ncep_pes10_null).reset_index()
df_ncep_pes10_null.columns = ['Feature', 'Null Count']  # Spalten umbenennen


# Ausgabe des neuen DataFrames
display(df_ncep_pes10_null)

def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        # Überprüfen, ob df ein DataFrame ist
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

        # Dateiname für den DataFrame
        png_filename = f"{df_name}.png"

        # Styling-Optionen definieren
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)

        # Vertikale und horizontale Linien für jede Zelle hinzufügen
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        # Dateipfad und -namen kombinieren
        png_full_path = f"{path}/{png_filename}"

        # DataFrame als Bild speichern mit dfi
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')

min_values = df_ncep_pes10.min()
max_values = df_ncep_pes10.max()

min_max_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values})
display(min_max_df)
# df_dwd_pes10_null.merge(min_max_df,on = 'Feature', how='inner')
merged_df_standard = pd.merge(df_ncep_pes10_null, min_max_df, on='Feature', how='inner')


merged_df_standard = merged_df_standard.T
merged_df_standard.columns = merged_df_standard.iloc[0]
merged_df_standard = merged_df_standard[1:]

expected_columns = ['Feature', 'ref_datetime','valid_datetime']
existing_columns = [col for col in expected_columns if col in merged_df_standard.columns]

merged_df_standard = merged_df_standard[existing_columns + [col for col in merged_df_standard.columns if col not in existing_columns]]

cols_to_convert = merged_df_standard.columns.difference(expected_columns)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].astype(float)
merged_df_standard[cols_to_convert] = merged_df_standard[cols_to_convert].round(2)


# Korrigierter DataFrame-Namen
dataframes = {
    'df_ncep_pes10_standard': merged_df_standard,  
}

export_styled_dataframes(dataframes, path=r'...')
display(merged_df_standard)


In [ ]:
100/22233420*104180

## DWD Solar

In [153]:
dwd_solar = pd.read_parquet(r"data\dwd_solar_processed.parquet")

In [ ]:
num_dwd_solar_cols = dwd_solar.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_dwd_solar_cols.columns)):
    plt.subplot(2, 2, i+1)
    sns.histplot(data=num_dwd_solar_cols, x=num_dwd_solar_cols.columns[i])
    plt.title(num_dwd_solar_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
num_dwd_solar_cols = dwd_solar.select_dtypes(include="number")
plt.figure(figsize= (12, 12))
for i in range(len(num_dwd_solar_cols.columns)):
    plt.subplot(2, 2, i+1)
    sns.boxplot(data=num_dwd_solar_cols, x=num_dwd_solar_cols.columns[i])
    plt.title(num_dwd_solar_cols.columns[i])
plt.tight_layout(pad=2)
plt.show()

In [ ]:
corr = dwd_solar.select_dtypes(include="number").corr()
sns.heatmap(corr, annot=True)

In [ ]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien/'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df_remit = pd.concat(dataframes, ignore_index=True)

# Konvertiere die Spalten 'publishTime' und 'createdTime' in Datetime-Objekte
df_remit['publishTime'] = pd.to_datetime(df_remit['publishTime'])
df_remit['createdTime'] = pd.to_datetime(df_remit['createdTime'])

# Versuche, 'eventStatus' und 'eventStartTime' in Datetime-Objekte zu konvertieren; ungültige Werte werden in NaT umgewandelt
df_remit['eventStatus'] = pd.to_datetime(df_remit['eventStatus'], errors='coerce')
df_remit['eventStartTime'] = pd.to_datetime(df_remit['eventStartTime'], errors='coerce')

start_date = '2020-09-20'
end_date = '2024-05-24'

# DataFrame filtern für den definierten Zeitraum
df_remit_filter = df_remit[(df_remit['createdTime'] >= start_date) & (df_remit['createdTime'] <= end_date)]
df_remit_h3 = df_remit_filter.head(3).round(2)

In [ ]:
df_remit_h3.columns

In [159]:
df_remit_h3_1 = df_remit_h3.filter(['id', 'dataset', 'mrid', 'revisionNumber', 'publishTime', 'createdTime', 'eventType','unavailabilityType', 
          ])

df_remit_h3_2 = df_remit_h3.filter(['id','participantId', 'registrationCode', 'assetId', 'assetType',
       'affectedUnit', 'affectedUnitEIC', 'affectedArea', 'biddingZone', 'normalCapacity', 'availableCapacity',
       'unavailableCapacity',])
df_remit_h3_3 = df_remit_h3.filter(['id',
       'unavailableCapacity', 'eventStatus', 'eventStartTime', 'eventEndTime',
       'cause', ])


In [ ]:
df_remit_h3_2

In [ ]:
df_remit.head()

In [ ]:
df_remit_h3_3

##### Erstellung der Standertwerte

In [163]:
def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
      
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue

      
        png_filename = f"{df_name}.png"

     
        styled_df = (df.style
            .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
            .set_table_styles(
                [{'selector': 'thead th', 
                  'props': [('background-color', 'darkblue'), 
                            ('color', 'white'), 
                            ('font-weight', 'bold'), 
                            ('border', '1px solid black')]}],
                axis=0
            )
            .set_properties(**{
                'border': '1px solid black',
                'text-align': 'center',
                'font-size': '14px'
            })
            .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
        )

        float_columns = df.select_dtypes(include=['float']).columns
        format_dict = {col: '{:.2f}' for col in float_columns}
        styled_df = df.style.format(format_dict)
        
        styled_df.set_table_styles(
            [{'selector': 'td, th', 
              'props': [('border', '1px solid black')]}],
            axis=0
        )

        
        png_full_path = f"{path}/{png_filename}"

       
        dfi.export(styled_df, png_full_path, table_conversion='matplotlib')




dataframes = {
    'df_energy_h3': energy_h3,  
    'df_dwd_hornsea_h3': df_hornsea_dwd_h3,  
    'df_dwd_pes10_h3': df_dwd_pes10_h3,  
    'df_dwd_demand_h3': df_dwd_demand_h3,  
    'df_ncep_hornsea_h3': df_ncep_hornsea_h3,  
    'df_ncep_pes10_h3': df_ncep_pes10_h3,  
    'df_ncep_demand_h3': df_ncep_demand_h3,  
    'df_remit_h3_1':df_remit_h3_1,
    'df_remit_h3_2':df_remit_h3_2,
    'df_remit_h3_3':df_remit_h3_3,
}

export_styled_dataframes(dataframes, path=r'...')



# Schluss

In [ ]:
df_viz = dwd_solar.loc["2022-11-15": "2023-01-15"]
px.line(df_viz, x=df_viz.index, y="SolarDownwardRadiation")

In [ ]:
df = pd.read_parquet(r"artifacts\prepared_data\merged_data.parquet")
df.head()

In [ ]:
data = pd.read_parquet(r"artifacts\prepared_data\merged_data.parquet")
data.head()

In [ ]:
ax = WindroseAxes.from_ax()
ax.bar(data['WindDirection:100'], data['WindSpeed:100'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title("Wind Direction (°) VS Wind Speed (m/s)")
plt.show()

##### Winddaten Ausschnitt

In [ ]:
px.line(df_hornsea_dwd, x="valid_time", y=["WindSpeed", "WindSpeed:100"])

In [364]:
df1 = df_hornsea_dwd
df_2 =df_hornsea_ncep

In [ ]:


# Annahme: num_dwd_hornsea_cols enthält die numerischen Spalten von df_hornsea_dwd
num_dwd_hornsea_cols = df_hornsea_dwd.select_dtypes(include="number")

# Erzeuge das 3x3 Raster für die Subplots
fig = sp.make_subplots(rows=3, cols=3, subplot_titles=num_dwd_hornsea_cols.columns)

# Schleife durch jede Spalte und füge ein Histogramm zu den Subplots hinzu
for i, column in enumerate(num_dwd_hornsea_cols.columns):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    hist = px.histogram(num_dwd_hornsea_cols, x=column)
    fig.add_trace(hist['data'][0], row=row, col=col)

# Layout-Anpassungen
fig.update_layout(height=800, width=800, title_text="Histogramme der numerischen Spalten", showlegend=False)
fig.show()


In [137]:
year_to_filter = 2023
df_dwd_pes10_2023 = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter]
df_ncep_pes10_2023 = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter]


In [503]:



# Numerische Spalten in beiden DataFrames auswählen
num_dwd_hornsea_cols_1 = df_ncep_pes10_2023.select_dtypes(include="number")
num_dwd_hornsea_cols_2 = df_dwd_pes10_2023.select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in num_dwd_hornsea_cols_1.columns.intersection(num_dwd_hornsea_cols_2.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    # Histogramm für den ersten DataFrame
    hist_1 = go.Histogram(x=num_dwd_hornsea_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
    fig.add_trace(hist_1, row=row, col=col)
    
    # Histogramm für den zweiten DataFrame
    hist_2 = go.Histogram(x=num_dwd_hornsea_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
    fig.add_trace(hist_2, row=row, col=col)

# Layout-Anpassungen
fig.update_layout(
    height=700,
    width=1200,
    title_text="Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10",
    showlegend=True,
    margin=dict(t=50, b=20, l=20, r=20)
)

# Den Abstand zwischen der letzten Reihe und dem Bild verringern
fig.update_yaxes(title_standoff=0)

pio.write_html(fig, 'historgramm.html', auto_open=True)  # Temporäre HTML-Datei erstellen

In [70]:




# richtig
year_to_filter = 2023
df_dwd_pes10_year = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter]
df_ncep_pes10_year = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter]



# Numerische Spalten in beiden DataFrames auswählen
num_dwd_hornsea_cols_1 = df_ncep_pes10_year.select_dtypes(include="number")
num_dwd_hornsea_cols_2 = df_dwd_pes10_year.select_dtypes(include="number")

# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in num_dwd_hornsea_cols_1.columns.intersection(num_dwd_hornsea_cols_2.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    # Histogramm für den ersten DataFrame
    hist_1 = go.Histogram(x=num_dwd_hornsea_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
    fig.add_trace(hist_1, row=row, col=col)
    
    # Histogramm für den zweiten DataFrame
    hist_2 = go.Histogram(x=num_dwd_hornsea_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
    fig.add_trace(hist_2, row=row, col=col)

# Layout-Anpassungen
fig.update_layout(
    height=700,
    width=1200,
    title_text=f"Histogramme der numerischen Spalten von DWD honrsea und honrsea pes10 von {year_to_filter}",
    showlegend=True,
    margin=dict(t=50, b=20, l=20, r=20)
)

# Den Abstand zwischen der letzten Reihe und dem Bild verringern
fig.update_yaxes(title_standoff=0)

pio.write_html(fig, r'historgramm_dwd_pes10_vs_ncep_pes10_{year_to_filter}.html', auto_open=True)  # Temporäre HTML-Datei erstellen

In [71]:


# Fertig

# Numerische Spalten in beiden DataFrames auswählen
num_dwd_hornsea_cols_1 = df_hornsea_ncep.select_dtypes(include="number")
num_dwd_hornsea_cols_2 = df_hornsea_dwd.select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in num_dwd_hornsea_cols_1.columns.intersection(num_dwd_hornsea_cols_2.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    # Histogramm für den ersten DataFrame
    hist_1 = go.Histogram(x=num_dwd_hornsea_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
    fig.add_trace(hist_1, row=row, col=col)
    
    # Histogramm für den zweiten DataFrame
    hist_2 = go.Histogram(x=num_dwd_hornsea_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
    fig.add_trace(hist_2, row=row, col=col)

# Layout-Anpassungen
fig.update_layout(
    height=700,
    width=1200,
    title_text=f"Histogramme der numerischen Spalten von DWD honrsea und NCEP honrsea",
    showlegend=True,
    margin=dict(t=50, b=20, l=20, r=20)
)

# Den Abstand zwischen der letzten Reihe und dem Bild verringern
fig.update_yaxes(title_standoff=0)

pio.write_html(fig, f'historgramm_dwd_hornsea_vs_ncep_honrsea.html', auto_open=True) 

In [72]:



# Liste der Jahre, die gefiltert werden sollen
years_to_filter = [2021, 2022,2023,2024]

# Schleife durch jedes Jahr
for year_to_filter in years_to_filter:
    df_dwd_pes10_year = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter]
    df_ncep_pes10_year = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter]


    # Numerische Spalten in beiden DataFrames auswählen
    num_dwd_cols_1 = df_dwd_pes10_year.select_dtypes(include="number")
    num_dwd_cols_2 = df_ncep_pes10_year.select_dtypes(include="number")


    # Spalten definieren, die ausgeschlossen werden sollen
    exclude_columns = ["hours_after"]

    # Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
    common_columns = [col for col in num_dwd_cols_1.columns.intersection(num_dwd_cols_2.columns) if col not in exclude_columns]

    # Erzeuge das 2x3 Raster für die Subplots
    fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

    # Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
    for i, column in enumerate(common_columns[:6]):
        row = (i // 3) + 1
        col = (i % 3) + 1
        
        # Histogramm für den ersten DataFrame
        hist_1 = go.Histogram(x=num_dwd_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
        fig.add_trace(hist_1, row=row, col=col)
        
        # Histogramm für den zweiten DataFrame
        hist_2 = go.Histogram(x=num_dwd_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
        fig.add_trace(hist_2, row=row, col=col)

    # Layout-Anpassungen
    fig.update_layout(
        height=700,
        width=1200,
        title_text=f"Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10 von {year_to_filter}",
        showlegend=True,
        margin=dict(t=50, b=20, l=20, r=20)
    )

    # Den Abstand zwischen der letzten Reihe und dem Bild verringern
    fig.update_yaxes(title_standoff=0)

    pio.write_html(fig, f'historgramm_dwd_pes10_vs_ncep_pes10_von_{year_to_filter}.html', auto_open=True) 

In [549]:


# Liste der Jahre, die gefiltert werden sollen
years_to_filter = [2021, 2022,2023,2024]

# Schleife durch jedes Jahr
for year_to_filter in years_to_filter:
    df_dwd_demand_year = df_dwd_demand[df_dwd_demand['ref_datetime'].dt.year == year_to_filter]
    df_ncep_demand_year = df_ncep_demand[df_ncep_demand['ref_datetime'].dt.year == year_to_filter]

    # Numerische Spalten in beiden DataFrames auswählen
    num_dwd_cols_1 = df_ncep_demand_year.select_dtypes(include="number")
    num_dwd_cols_2 = df_dwd_demand_year.select_dtypes(include="number")

    # Spalten definieren, die ausgeschlossen werden sollen
    exclude_columns = ["hours_after"]

    # Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
    common_columns = [col for col in num_dwd_cols_1.columns.intersection(num_dwd_cols_2.columns) if col not in exclude_columns]

    # Erzeuge das 2x3 Raster für die Subplots
    fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

    # Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
    for i, column in enumerate(common_columns[:6]):
        row = (i // 3) + 1
        col = (i % 3) + 1

        # Histogramm für den ersten DataFrame
        hist_1 = go.Histogram(x=num_dwd_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
        fig.add_trace(hist_1, row=row, col=col)

        # Histogramm für den zweiten DataFrame
        hist_2 = go.Histogram(x=num_dwd_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
        fig.add_trace(hist_2, row=row, col=col)

    # Layout-Anpassungen
    fig.update_layout(
        height=700,
        width=1200,
        title_text=f"Histogramme der numerischen Spalten von DWD demand und NCEP demand von {year_to_filter}",
        showlegend=True,
        margin=dict(t=50, b=20, l=20, r=20)
    )

    # Den Abstand zwischen der letzten Reihe und dem Bild verringern
    fig.update_yaxes(title_standoff=0)

    pio.write_html(fig, f'historgramm_dwd_demand_vs_ncep_demand_von_{year_to_filter}.html', auto_open=True)


In [ ]:
df_dwd_pes10_year.dtypes

In [177]:
# df_dwd_demand

In [157]:
year_to_filter = 2023
df_dwd_demand_2024 = df_dwd_demand[df_dwd_demand['ref_datetime'].dt.year == year_to_filter]
df_ncep_demand_2024 = df_ncep_demand[df_ncep_demand['ref_datetime'].dt.year == year_to_filter]


In [ ]:


# Numerische Spalten in beiden DataFrames auswählen
num_dwd_hornsea_cols_1 = df_dwd_demand_2024.select_dtypes(include="number")
num_dwd_hornsea_cols_2 = df_ncep_demand_2024.select_dtypes(include="number")

# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in num_dwd_hornsea_cols_1.columns.intersection(num_dwd_hornsea_cols_2.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    row = (i // 3) + 1
    col = (i % 3) + 1
    
    # Histogramm für den ersten DataFrame
    hist_1 = go.Histogram(x=num_dwd_hornsea_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
    fig.add_trace(hist_1, row=row, col=col)
    
    # Histogramm für den zweiten DataFrame
    hist_2 = go.Histogram(x=num_dwd_hornsea_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
    fig.add_trace(hist_2, row=row, col=col)

# Layout-Anpassungen
fig.update_layout(
    height=700,
    width=1200,
    title_text="Histogramme der numerischen Spalten aus beiden DataFrames",
    showlegend=True,
    margin=dict(t=50, b=20, l=20, r=20)
)

# Den Abstand zwischen der letzten Reihe und dem Bild verringern
fig.update_yaxes(title_standoff=0)

pio.write_html(fig, 'temp_demand.html', auto_open=True)  # Temporäre HTML-Datei erstellen

In [ ]:
# df_dwd_demand_2024.groupby(['latitude','longitude'])['RelativeHumidity'].unique()

unique_values_list = df_dwd_demand_2024.groupby(['latitude','longitude'])['RelativeHumidity'].agg(lambda x: list(x.unique()))

print(unique_values_list)


In [ ]:
unique_values_list = df_ncep_demand_2024.groupby(['latitude','longitude'])['RelativeHumidity'].agg(lambda x: list(x.unique()))

print(unique_values_list)

# Remit-Datei

In [ ]:
df_remit.tail()

In [ ]:
df_remit_filter_h3 = df_remit_filter.head()
df_remit_filter_h3

In [203]:


def export_styled_dataframes(dfs_dict, path='.'):
    for df_name, df in dfs_dict.items():
        if not isinstance(df, pd.DataFrame):
            print(f"'{df_name}' ist kein DataFrame. Überspringe diesen Eintrag.")
            continue
        
        # Überprüfen, ob df_name 'df_remit_h3' ist und DataFrame aufteilen
        if df_name == 'df_remit_h3':
            # Angenommen, wir teilen df_remit_h3 in zwei Hälften
            mid_index = len(df) // 2
            df_part1 = df.iloc[:mid_index]  # Erste Hälfte
            df_part2 = df.iloc[mid_index:]   # Zweite Hälfte

            # Export der ersten Hälfte
            export_single_dataframe(f"{df_name}_part1", df_part1, path)
            # Export der zweiten Hälfte
            export_single_dataframe(f"{df_name}_part2", df_part2, path)

        else:
            # Exportiere andere DataFrames
            export_single_dataframe(df_name, df, path)

def export_single_dataframe(df_name, df, path):
    png_filename = f"{df_name}.png"
    
    styled_df = (df.style
        .set_table_attributes('style="width: 100%; border-collapse: collapse;"')
        .set_table_styles(
            [{'selector': 'thead th', 
              'props': [('background-color', 'darkblue'), 
                        ('color', 'white'), 
                        ('font-weight', 'bold'), 
                        ('border', '1px solid black')]}],
            axis=0
        )
        .set_properties(**{
            'border': '1px solid black',
            'text-align': 'center',
            'font-size': '14px'
        })
        .apply(lambda x: ['background-color: lightgrey' if i % 2 == 0 else '' for i in range(len(x))], axis=0)
    )

    float_columns = df.select_dtypes(include=['float']).columns
    format_dict = {col: '{:.2f}' for col in float_columns}
    styled_df = styled_df.format(format_dict)
    
    styled_df.set_table_styles(
        [{'selector': 'td, th', 
          'props': [('border', '1px solid black')]}],
        axis=0
    )

    png_full_path = f"{path}/{png_filename}"
    
    # Exportieren des DataFrames
    dfi.export(styled_df, png_full_path, table_conversion='matplotlib')


# Dictionary der DataFrames
dataframes = {
    'df_energy_h3': energy_h3,  
    'df_dwd_hornsea_h3': df_hornsea_dwd_h3,  
    'df_dwd_pes10_h3': df_dwd_pes10_h3,  
    'df_dwd_demand_h3': df_dwd_demand_h3,  
    'df_ncep_hornsea_h3': df_ncep_hornsea_h3,  
    'df_ncep_pes10_h3': df_ncep_pes10_h3,  
    'df_ncep_demand_h3': df_ncep_demand_h3,  
    'df_remit_h3_1': df_remit_h3_1,
    'df_remit_h3_2': df_remit_h3_2,
}

export_styled_dataframes(dataframes, path=r'...')


In [474]:
df_remit_t = df_remit

In [ ]:

# Funktion zum Extrahieren der Werte aus dem Dictionary
def extract_values(outage_profile):
    # Überprüfen, ob der Wert in der Spalte gültig ist
    if isinstance(outage_profile, str):
        # Suchen nach allen startTime, endTime und capacity Werten
        start_times = re.findall(r"'startTime': '(.*?)'", outage_profile)
        end_times = re.findall(r"'endTime': '(.*?)'", outage_profile)
        capacities = re.findall(r"'capacity': (\d+)", outage_profile)
        
        # Erstellen eines Dictionaries mit den Werten in nebeneinander liegenden Spalten
        extracted_data = {}
        for i in range(len(start_times)):
            extracted_data[f'startTime_{i+1}'] = start_times[i]
            extracted_data[f'endTime_{i+1}'] = end_times[i]
            extracted_data[f'capacity_{i+1}'] = capacities[i]

        return extracted_data
    else:
        return {}

# Extrahieren der Werte aus der 'outageProfile'-Spalte
extracted_values = df_remit_t['outageProfile'].apply(extract_values)

# Erstellen eines neuen DataFrames aus den extrahierten Werten
extracted_df = pd.DataFrame(extracted_values.tolist())

# Umwandlung der Spalten in den entsprechenden Datentyp
for col in extracted_df.columns:
    if 'startTime' in col or 'endTime' in col:
        extracted_df[col] = pd.to_datetime(extracted_df[col], errors='coerce')  # Fehlerhafte Einträge werden zu NaT
    elif 'capacity' in col:
        extracted_df[col] = extracted_df[col].astype(float)  # Oder int, je nach Bedarf

display(extracted_df)


In [ ]:
# Konvertiere die Spalten in Datetime-Objekte
for col in extracted_df.columns:
    if 'startTime' in col or 'endTime' in col:
        extracted_df[col] = pd.to_datetime(extracted_df[col])

# Berechnung der Zeitdifferenzen für alle Zeitpaare
for i in range(1, len(df.columns)//2 + 1):
    start_col = f'startTime_{i}'
    end_col = f'endTime_{i}'
    time_col = f'Time_{i}'
    
    if start_col in extracted_df.columns and end_col in extracted_df.columns:
        extracted_df[time_col] = extracted_df[end_col] - extracted_df[start_col]

extracted_df

In [ ]:
min_values = extracted_df.min()
max_values = extracted_df.max()
mean_values = extracted_df.mean()
median_values = extracted_df.median()

min_max_df_extracted_df = pd.DataFrame({'Feature': min_values.index, 'Min': min_values.values, 'Max': max_values.values,'Mean': mean_values.values, 'Median': median_values.values})
display(min_max_df_extracted_df)

In [65]:


# Daten filtern und numerische Spalten auswählen
df_ncep_pes10_year = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")
df_dwd_pes10_year = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_dwd_pes10_year.columns.intersection(df_ncep_pes10_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_dwd_pes10_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue",stacked=True)
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_ncep_pes10_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red", )
    
    ax.set_title(column)

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_pes10_vs_ncep_pes10.png')
plt.close()

In [78]:


# Daten filtern und numerische Spalten auswählen
df_ncep_pes10_year = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")
df_dwd_pes10_year = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")

# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_dwd_pes10_year.columns.intersection(df_ncep_pes10_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame (DWD)
    ax.hist(df_dwd_pes10_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue")
    
    # Histogramm für den zweiten DataFrame (NCEP)
    ax.hist(df_ncep_pes10_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red")
    
    ax.set_title(column)

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_pes10_vs_ncep_pes10.png')
plt.close()


In [ ]:

np.random.seed(19680801)

n_bins = 10
x = np.random.randn(1000, 3)

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)

colors = ['red', 'tan', 'lime']
ax0.hist(x, n_bins, density=True, histtype='bar', color=colors, label=colors)
ax0.legend(prop={'size': 10})
ax0.set_title('bars with legend')

ax1.hist(x, n_bins, density=True, histtype='bar', stacked=True)
ax1.set_title('stacked bar')

ax2.hist(x, n_bins, histtype='step', stacked=True, fill=False)
ax2.set_title('stack step (unfilled)')

# Make a multiple-histogram of data-sets with different length.
x_multi = [np.random.randn(n) for n in [10000, 5000, 2000]]
ax3.hist(x_multi, n_bins, histtype='bar')
ax3.set_title('different sample sizes')

fig.tight_layout()
plt.show()

In [58]:


# Daten filtern und numerische Spalten auswählen
df_ncep_pes10_year = df_ncep_pes10[df_ncep_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")
df_dwd_pes10_year = df_dwd_pes10[df_dwd_pes10['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_dwd_pes10_year.columns.intersection(df_ncep_pes10_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_dwd_pes10_year[column], bins=100, alpha=0.5, label=f"DWD", color="black",density=True, stacked=True)
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_ncep_pes10_year[column], bins=100, alpha=0.5, label=f"NCEP", color="red",density=True, stacked=True)
    
    ax.set_title(column)
    ax.set_yscale('log')

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_pes10_vs_ncep_pes10_log.png')
plt.close()

In [582]:


# Daten filtern und numerische Spalten auswählen
df_hornsea_ncep_year = df_hornsea_ncep[df_hornsea_ncep['reference_time'].dt.year == year_to_filter].select_dtypes(include="number")
df_hornsea_dwd_year = df_hornsea_dwd[df_hornsea_dwd['reference_time'].dt.year == year_to_filter].select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_hornsea_dwd_year.columns.intersection(df_hornsea_ncep_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_hornsea_dwd_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue")
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_hornsea_ncep_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red")
    
    ax.set_title(column)

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD hornsea und NCEP hornsea")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_hornsea_vs_ncep_hornsea.png')
plt.close()

In [39]:


# Daten filtern und numerische Spalten auswählen
df_hornsea_ncep_year = df_hornsea_ncep[df_hornsea_ncep['reference_time'].dt.year == year_to_filter].select_dtypes(include="number")
df_hornsea_dwd_year = df_hornsea_dwd[df_hornsea_dwd['reference_time'].dt.year == year_to_filter].select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_hornsea_dwd_year.columns.intersection(df_hornsea_ncep_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_hornsea_dwd_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue")
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_hornsea_ncep_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red")
    
    ax.set_title(column)
    ax.set_yscale('log')
# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD hornsea und NCEP hornsea")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_hornsea_vs_ncep_hornsea_log.png')
plt.close()

In [583]:


# Daten filtern und numerische Spalten auswählen
df_ncep_demand_year = df_ncep_demand[df_ncep_demand['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")
df_dwd_demand_year = df_dwd_demand[df_dwd_demand['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")


# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_dwd_demand_year.columns.intersection(df_ncep_demand_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_dwd_demand_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue")
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_ncep_demand_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red")
    
    ax.set_title(column)

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD demand und NCEP demand")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_deamnd_vs_ncep_demand.png')
plt.close()

In [11]:
# Liste der Jahre, die gefiltert werden sollen
years_to_filter = [2021, 2022, 2023, 2024]

# Schleife durch jedes Jahr und Halbjahr
for year_to_filter in years_to_filter:
    for half in [1, 2]:  # 1: Erstes Halbjahr, 2: Zweites Halbjahr
        # Filter für das jeweilige Halbjahr anwenden
        if half == 1:
            df_dwd_pes10_half = df_dwd_pes10[(df_dwd_pes10['ref_datetime'].dt.year == year_to_filter) &
                                             (df_dwd_pes10['ref_datetime'].dt.month <= 6)]
            df_ncep_pes10_half = df_ncep_pes10[(df_ncep_pes10['ref_datetime'].dt.year == year_to_filter) &
                                               (df_ncep_pes10['ref_datetime'].dt.month <= 6)]
        else:
            df_dwd_pes10_half = df_dwd_pes10[(df_dwd_pes10['ref_datetime'].dt.year == year_to_filter) &
                                             (df_dwd_pes10['ref_datetime'].dt.month > 6)]
            df_ncep_pes10_half = df_ncep_pes10[(df_ncep_pes10['ref_datetime'].dt.year == year_to_filter) &
                                               (df_ncep_pes10['ref_datetime'].dt.month > 6)]

        # Numerische Spalten in beiden DataFrames auswählen
        num_dwd_cols_1 = df_dwd_pes10_half.select_dtypes(include="number")
        num_dwd_cols_2 = df_ncep_pes10_half.select_dtypes(include="number")

        # Spalten definieren, die ausgeschlossen werden sollen
        exclude_columns = ["hours_after"]

        # Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
        common_columns = [col for col in num_dwd_cols_1.columns.intersection(num_dwd_cols_2.columns) if col not in exclude_columns]

        # Erzeuge das 2x3 Raster für die Subplots
        fig = sp.make_subplots(rows=2, cols=3, subplot_titles=common_columns[:6], vertical_spacing=0.1)

        # Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
        for i, column in enumerate(common_columns[:6]):
            row = (i // 3) + 1
            col = (i % 3) + 1
            
            # Histogramm für den ersten DataFrame
            hist_1 = go.Histogram(x=num_dwd_cols_1[column], opacity=0.5, name=f"DWD: {column}", marker=dict(color="blue"))
            fig.add_trace(hist_1, row=row, col=col)
            
            # Histogramm für den zweiten DataFrame
            hist_2 = go.Histogram(x=num_dwd_cols_2[column], opacity=0.5, name=f"NCEP: {column}", marker=dict(color="red"))
            fig.add_trace(hist_2, row=row, col=col)

        # Layout-Anpassungen
        fig.update_layout(
            height=700,
            width=1200,
            title_text=f"Histogramme der numerischen Spalten von DWD pes10 und NCEP pes10 von {year_to_filter}, Halbjahr {half}",
            showlegend=True,
            margin=dict(t=50, b=20, l=20, r=20)
        )

        # Den Abstand zwischen der letzten Reihe und dem Bild verringern
        fig.update_yaxes(title_standoff=0)

        # Speichern des Diagramms als HTML-Datei
        pio.write_html(fig, f'histogramm_dwd_pes10_vs_ncep_pes10_von_{year_to_filter}_halbjahr_{half}.html', auto_open=True)


In [30]:
# Daten filtern und numerische Spalten auswählen
df_ncep_demand_year = df_ncep_demand[df_ncep_demand['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")
df_dwd_demand_year = df_dwd_demand[df_dwd_demand['ref_datetime'].dt.year == year_to_filter].select_dtypes(include="number")

# Spalten definieren, die ausgeschlossen werden sollen
exclude_columns = ["hours_after"]

# Gemeinsame Spalten identifizieren und die auszuschließenden Spalten entfernen
common_columns = [col for col in df_dwd_demand_year.columns.intersection(df_ncep_demand_year.columns) if col not in exclude_columns]

# Erzeuge das 2x3 Raster für die Subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 7))
axes = axes.flatten()

# Schleife durch jede Spalte und füge Histogramme beider DataFrames hinzu
for i, column in enumerate(common_columns[:6]):
    ax = axes[i]
    
    # Histogramm für den ersten DataFrame
    ax.hist(df_dwd_demand_year[column], bins=100, alpha=0.5, label=f"DWD: {column}", color="blue")
    
    # Histogramm für den zweiten DataFrame
    ax.hist(df_ncep_demand_year[column], bins=100, alpha=0.5, label=f"NCEP: {column}", color="red")
    
    ax.set_title(column)
    ax.set_yscale('log')  # Setze die y-Achse auf logarithmische Skala

# Legende außerhalb der Subplots platzieren
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1))

# Layout-Anpassungen
fig.suptitle(f"Histogramme der numerischen Spalten von DWD demand und NCEP demand")
fig.tight_layout(rect=[0, 0, 1, 0.96])

# Speichern der Grafik als PNG-Datei
plt.savefig(f'histogramm_dwd_deamnd_vs_ncep_demand_log.png')
plt.close()


In [ ]:
df_ncep_pes10.groupby(['CloudCover']).count().head(63)

In [ ]:
df_dwd_demand.tail()

In [ ]:
df_dwd_demand

In [ ]:
df_dwd_demand['name'] = 'dwd'
df_ncep_demand['name'] = 'ncep'

test_demand = pd.concat([df_dwd_demand, df_ncep_demand, ])

test_demand = test_demand.dropna()         
           

test_demand

In [340]:
test_demand = test_demand.dropna()

In [ ]:
print(test_demand['name'].duplicated().sum())


In [215]:
test_demand = test_demand.dropna()

In [223]:
test_demand = test_demand.dropna()

In [106]:
first_1000 = test_demand.head(1000)   # Erste 1000 Zeilen
last_1000 = test_demand.tail(1000)     # Letzte 1000 Zeilen

# Kombiniere die beiden DataFrames
combined_df = pd.concat([first_1000, last_1000], ignore_index=True)

In [ ]:
combined_df.isnull().sum()

## Grafiken vergleich der beiden Datensätz DWD vs NCEP

In [ ]:
df_dwd_demand['name'] = 'dwd'
df_ncep_demand['name'] = 'ncep'

df_demand_ncep_hornsea = pd.concat([df_dwd_demand, df_ncep_demand])

df_demand_ncep_hornsea = df_demand_ncep_hornsea.dropna()         
           

df_demand_ncep_hornsea.head()

In [ ]:
test_demand.isnull().sum()

In [ ]:
test_demand.shape

In [ ]:
# first_1000 = test_demand.head(20000000)   # Erste 1000 Zeilen
# last_1000 = test_demand.tail(20000000)     # Letzte 1000 Zeilen

# Kombiniere die beiden DataFrames
combined_df11 = test_demand.head(10813880)
combined_df11.shape

In [ ]:
combined_df11

In [ ]:


# Verfügbare Spalten im DataFrame
available_variables = combined_df11.columns.tolist()
print("Available variables:", available_variables)

# Liste der Variablen und ihrer Namen, nur wenn sie im DataFrame existieren
variables = ['point', 'RelativeHumidity', 'Temperature', 'TotalPrecipitation', 'WindDirection', 'WindSpeed']
valid_variables = [var for var in variables if var in available_variables]

# Dynamisches Einlesen der Gruppen
names = combined_df11['name'].unique()

# Setze den Stil für die Plots
sns.set(style="whitegrid")

# Berechne die Anzahl der benötigten Zeilen
num_cols = 3  # Maximal 3 Spalten
num_rows = (len(valid_variables) + num_cols - 1) // num_cols  # Runden auf die nächste ganze Zahl

# Erstelle Subplots für die Distributionsdiagramme
fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6 * num_rows))
axs = axs.flatten()  # Mache axs zu einem eindimensionalen Array

# Definiere die Einheiten für jede Variable
units = {
    'point': '',
    'RelativeHumidity': '%',
    'Temperature': '°C',
    'TotalPrecipitation': 'mm',
    'WindDirection': '°',
    'WindSpeed': 'm/s'
}

for i, variable in enumerate(valid_variables):
    for name in names:
        # Filtere die Daten für die jeweilige Gruppe
        filtered_data = combined_df11[combined_df11['name'] == name]
        
        # Überprüfen, ob gefilterte Daten vorhanden sind
        if filtered_data.empty:
            # Ausgabe nur für Gruppen, die keine Daten haben
            continue
        
        # Erstelle das Distributionsdiagramm für jede Gruppe
        sns.kdeplot(
            data=filtered_data,
            x=variable,  
            fill=True,
            ax=axs[i],
            alpha=0.5,
            label=name  # Gruppennamen für die Legende hinzufügen
        )
    
    # Titel und Achsenbeschriftungen hinzufügen
    axs[i].set_title(f'Distribution of {variable}', fontsize=16)
    
    # Einheit zur x-Achsenbeschriftung hinzufügen, überspringe 'point'
    if variable != 'point':
        axs[i].set_xlabel(f'{variable} ({units[variable]})')
    else:
        axs[i].set_xlabel(variable)  # Nur den Namen ohne Einheit

    axs[i].set_ylabel('Density')
    axs[i].legend(title='Group')  # Legende hinzufügen

    # Ganzzahlige Werte auf der Y-Achse einstellen
    axs[i].yaxis.set_major_locator(MaxNLocator(integer=True))

# Leere Subplots ausblenden, falls weniger als num_rows*num_cols Variablen vorhanden sind
for j in range(i + 1, num_rows * num_cols):
    fig.delaxes(axs[j])

# Layout anpassen
plt.tight_layout()

# Optional: Speichere das Diagramm oder zeige es an
plt.savefig('distribution_plots_demand.png')
plt.show()


In [ ]:
df_hornsea_dwd['name'] = 'dwd'
df_hornsea_ncep['name'] = 'ncep'

df_dwd_ncep_hornsea = pd.concat([df_hornsea_dwd, df_hornsea_ncep])

df_dwd_ncep_hornsea = df_dwd_ncep_hornsea.dropna()         
           

df_dwd_ncep_hornsea.head()

In [ ]:
# Verfügbare Spalten im DataFrame
available_variables = df_dwd_ncep_hornsea.columns.tolist()
print("Available variables:", available_variables)

# Liste der Variablen und ihrer Namen, nur wenn sie im DataFrame existieren
variables = ['RelativeHumidity', 'Temperature', 'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100']
valid_variables = [var for var in variables if var in available_variables]

# Dynamisches Einlesen der Gruppen
names = df_dwd_ncep_hornsea['name'].unique()

sns.set(style="whitegrid")

# Berechne die Anzahl der benötigten Zeilen
num_cols = 3  # Maximal 3 Spalten
num_rows = (len(valid_variables) + num_cols - 1) // num_cols  # Runden auf die nächste ganze Zahl

# Erstelle Subplots für die Distributionsdiagramme
fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6 * num_rows))
axs = axs.flatten()  # Mache axs zu einem eindimensionalen Array

for i, variable in enumerate(valid_variables):
    for name in names:
        # Filtere die Daten für die jeweilige Gruppe
        filtered_data = df_dwd_ncep_hornsea[df_dwd_ncep_hornsea['name'] == name]
        
        # Überprüfen, ob gefilterte Daten vorhanden sind
        if filtered_data.empty:
            print(f"No data available for group {name}.")
            continue
        
        # Erstelle das Distributionsdiagramm für jede Gruppe
        sns.kdeplot(
            data=filtered_data,
            x=variable,  
            fill=True,
            ax=axs[i],
            alpha=0.5,
            label=name  # Gruppennamen für die Legende hinzufügen
        )
    
    # Titel und Achsenbeschriftungen hinzufügen
    axs[i].set_title(f'Distribution of {variable}', fontsize=16)
    axs[i].set_xlabel(variable)
    axs[i].set_ylabel('Density')
    axs[i].legend(title='Group')  # Legende hinzufügen

    # Ganzzahlige Werte auf der Y-Achse einstellen
    axs[i].yaxis.set_major_locator(MaxNLocator(integer=True))

# Leere Subplots ausblenden, falls weniger als num_rows*num_cols Variablen vorhanden sind
for j in range(i + 1, num_rows * num_cols):
    fig.delaxes(axs[j])

# Layout anpassen
plt.tight_layout()

# Optional: Speichere das Diagramm oder zeige es an
plt.savefig('distribution_plots_hornsea.png')
plt.show()


In [ ]:



# Verfügbare Spalten im DataFrame
available_variables = df_dwd_ncep_hornsea.columns.tolist()
print("Available variables:", available_variables)

# Liste der Variablen und ihrer Namen, nur wenn sie im DataFrame existieren
variables = ['RelativeHumidity', 'Temperature', 'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100']
valid_variables = [var for var in variables if var in available_variables]

# Dynamisches Einlesen der Gruppen
names = df_dwd_ncep_hornsea['name'].unique()

# Setze den Stil für die Plots
sns.set(style="whitegrid")

# Berechne die Anzahl der benötigten Zeilen
num_cols = 3  # Maximal 3 Spalten
num_rows = (len(valid_variables) + num_cols - 1) // num_cols  # Runden auf die nächste ganze Zahl

# Erstelle Subplots für die Distributionsdiagramme
fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6 * num_rows))
axs = axs.flatten()  # Mache axs zu einem eindimensionalen Array

# Definiere die Einheiten für jede Variable
units = {
    'RelativeHumidity': '%',
    'Temperature': '°C',
    'WindDirection': '°',
    'WindDirection:100': '°',
    'WindSpeed': 'm/s',
    'WindSpeed:100': 'm/s'
}

for i, variable in enumerate(valid_variables):
    for name in names:
        # Filtere die Daten für die jeweilige Gruppe
        filtered_data = df_dwd_ncep_hornsea[df_dwd_ncep_hornsea['name'] == name]
        
        # Überprüfen, ob gefilterte Daten vorhanden sind
        if filtered_data.empty:
            print(f"No data available for group {name}.")
            continue
        
        # Erstelle das Distributionsdiagramm für jede Gruppe
        sns.kdeplot(
            data=filtered_data,
            x=variable,  
            fill=True,
            ax=axs[i],
            alpha=0.5,
            label=name  # Gruppennamen für die Legende hinzufügen
        )
    
    # Titel und Achsenbeschriftungen hinzufügen
    axs[i].set_title(f'Distribution of {variable}', fontsize=16)
    axs[i].set_xlabel(f'{variable} ({units[variable]})')  # Einheit zur x-Achsenbeschriftung hinzufügen
    axs[i].set_ylabel('Density')
    axs[i].legend(title='Group')  # Legende hinzufügen

    # Ganzzahlige Werte auf der Y-Achse einstellen
    axs[i].yaxis.set_major_locator(MaxNLocator(integer=True))

    # Maximalwert für die x-Achse einstellen
    max_value = filtered_data[variable].max()
    axs[i].set_xlim(0, max_value * 1.1)  # Setze das Maximum auf 10% über dem Maximalwert

# Leere Subplots ausblenden, falls weniger als num_rows*num_cols Variablen vorhanden sind
for j in range(i + 1, num_rows * num_cols):
    fig.delaxes(axs[j])

# Layout anpassen
plt.tight_layout()

# Optional: Speichere das Diagramm oder zeige es an
plt.savefig('distribution_plots_hornsea.png')
plt.show()


pes 10

In [ ]:
df_dwd_pes10['name'] = 'dwd'
df_ncep_pes10['name'] = 'ncep'

df_dwd_ncep_pes10 = pd.concat([df_dwd_pes10, df_ncep_pes10])

df_dwd_ncep_pes10 = df_dwd_ncep_pes10.dropna()         
           
df_dwd_ncep_pes10.head()

In [ ]:
# Verfügbare Spalten im DataFrame
available_variables = df_dwd_ncep_pes10.columns.tolist()
print("Available variables:", available_variables)

# Liste der Variablen und ihrer Namen, nur wenn sie im DataFrame existieren
variables = ['point', 'CloudCover', 'SolarDownwardRadiation', 'Temperature', 'latitude', 'longitude']
valid_variables = [var for var in variables if var in available_variables]

# Dynamisches Einlesen der Gruppen
names = df_dwd_ncep_pes10['name'].unique()

sns.set(style="whitegrid")

# Berechne die Anzahl der benötigten Zeilen
num_cols = 3  # Maximal 3 Spalten
num_rows = (len(valid_variables) + num_cols - 1) // num_cols  # Runden auf die nächste ganze Zahl

# Erstelle Subplots für die Distributionsdiagramme
fig, axs = plt.subplots(num_rows, num_cols, figsize=(18, 6 * num_rows))
axs = axs.flatten()  # Mache axs zu einem eindimensionalen Array

# Definiere die Einheiten für jede Variable
units = {
    'point': '',
    'CloudCover': '%',
    'SolarDownwardRadiation': 'W/m²',
    'Temperature': '°C',
    'latitude': '',
    'longitude': ''
}

for i, variable in enumerate(valid_variables):
    for name in names:
        # Filtere die Daten für die jeweilige Gruppe
        filtered_data = df_dwd_ncep_pes10[df_dwd_ncep_pes10['name'] == name]
        
        # Überprüfen, ob gefilterte Daten vorhanden sind
        if filtered_data.empty:
            continue  # Keine Ausgabe, wenn keine Daten vorhanden sind
        
        # Erstelle das Distributionsdiagramm für jede Gruppe
        sns.kdeplot(
            data=filtered_data,
            x=variable,  
            fill=True,
            ax=axs[i],
            alpha=0.5,
            label=name  # Gruppennamen für die Legende hinzufügen
        )
    
    # Titel und Achsenbeschriftungen hinzufügen
    axs[i].set_title(f'Distribution of {variable}', fontsize=16)
    
    # Einheit zur x-Achsenbeschriftung hinzufügen, überspringe 'point', 'latitude' und 'longitude'
    if variable not in ['point', 'latitude', 'longitude']:
        axs[i].set_xlabel(f'{variable} ({units[variable]})')
    else:
        axs[i].set_xlabel(variable)  # Nur den Namen ohne Einheit

    axs[i].set_ylabel('Density')
    axs[i].legend(title='Group')  # Legende hinzufügen

    # Ganzzahlige Werte auf der Y-Achse einstellen
    axs[i].yaxis.set_major_locator(MaxNLocator(integer=True))

# Leere Subplots ausblenden, falls weniger als num_rows*num_cols Variablen vorhanden sind
for j in range(i + 1, num_rows * num_cols):
    fig.delaxes(axs[j])

# Layout anpassen
plt.tight_layout()

# Optional: Speichere das Diagramm oder zeige es an
plt.savefig('distribution_plots_pes10.png')
plt.show()


In [ ]:


def plot_histograms(combined_df11, variables, group_col='name', max_cols=3):
    # Verfügbare Spalten im DataFrame
    available_variables = combined_df11.columns.tolist()
    print("Available variables:", available_variables)

    # Liste der Variablen und ihrer Namen, nur wenn sie im DataFrame existieren
    valid_variables = [var for var in variables if var in available_variables]

    # Dynamisches Einlesen der Gruppen
    names = combined_df11[group_col].unique()

    sns.set(style="whitegrid")

    # Berechne die Anzahl der benötigten Zeilen
    num_rows = (len(valid_variables) + max_cols - 1) // max_cols  # Runden auf die nächste ganze Zahl

    # Erstelle Subplots für die Histogramme
    fig, axs = plt.subplots(num_rows, max_cols, figsize=(18, 6 * num_rows))
    axs = axs.flatten()  # Mache axs zu einem eindimensionalen Array

    for i, variable in enumerate(valid_variables):
        for name in names:
            # Filtere die Daten für die jeweilige Gruppe
            filtered_data = combined_df11[combined_df11[group_col] == name]
            
            # Überprüfen, ob gefilterte Daten vorhanden sind
            if filtered_data.empty:
                print(f"No data available for group {name}.")
                continue
            
            # Erstelle ein Histogramm für jede Gruppe
            sns.histplot(
                data=filtered_data,
                x=variable,
                ax=axs[i],
                bins=30,  # Anzahl der Bins kann angepasst werden
                stat='count',  # Zähle die Anzahl der Beobachtungen in jedem Bin
                alpha=0.5,
                label=name,  # Gruppennamen für die Legende hinzufügen
                kde=False  # Dichtefunktion ausblenden
            )
        
        # Titel und Achsenbeschriftungen hinzufügen
        axs[i].set_title(f'Distribution of {variable}', fontsize=16)
        axs[i].set_xlabel(variable)
        axs[i].set_ylabel('Count')  # y-Achsenbezeichnung auf 'Count' ändern
        axs[i].legend(title='Group')  # Legende hinzufügen

        # Ganzzahlige Werte auf der Y-Achse einstellen
        axs[i].yaxis.set_major_locator(MaxNLocator(integer=True))

    # Leere Subplots ausblenden, falls weniger als num_rows*num_cols Variablen vorhanden sind
    for j in range(i + 1, num_rows * max_cols):
        fig.delaxes(axs[j])

    # Layout anpassen
    plt.tight_layout()

    # Optional: Speichere das Diagramm oder zeige es an
    plt.savefig('histogram_plots_demand.png')
    plt.show()

# Beispielaufruf der Funktion
variables = ['point', 'RelativeHumidity', 'Temperature', 'TotalPrecipitation', 'WindDirection', 'WindSpeed']
plot_histograms(combined_df11, variables)
